# Download USGS flow data
We use this script to first download the data availability info for each USGS station, and second to download the data for our period of interest. 

In [1]:
import sys
import pandas as pd
from pathlib import Path
#import dataretrieval.nwis as nwis
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
data_path     = cs.read_from_config(config_file,'data_path')

# CAMELS-spat metadata
cs_meta_path  = cs.read_from_config(config_file,'cs_basin_path')
cs_meta_name  = cs.read_from_config(config_file,'cs_meta_name')

# Basin folder
cs_basin_folder = cs.read_from_config(config_file, 'cs_basin_path')

# Data period
time_s = cs.read_from_config(config_file, 'usgs_start_t')
time_e = cs.read_from_config(config_file, 'usgs_start_e')

### Data loading

In [4]:
# CAMELS-spat metadata file
cs_meta_path = Path(data_path) / cs_meta_path
cs_meta = pd.read_csv(cs_meta_path / cs_meta_name)

### Loop over sites and download the flow record

In [ ]:
# TO DO
# 2. For each site, 
# 2a. Download data
# 2b. Remove the top row(s) that are not comments (leading #) but also not data

# NEXT SCRIPT
# 2c. Update meta-data file with 'record_start', 'record_end', 'record_missing', 'record_native_time_resolution' fields
# 2d. Convert local time to ... whatever ERA5 is in (UTC, probably?)
# 2e. Resample to hourly, but save raw data as well (usgs_site_raw_qobs.csv, usgs_site_hourly_qobs.csv) - we can decide later

In [5]:
# General settings
var = '00060' # streamflow; 00065 for gage height
main_url = 'https://nwis.waterservices.usgs.gov/nwis/iv/' # (i)nstantaneous (v)alues

In [21]:
# Loop over the USA stations only
for ix,row in cs_meta.iterrows():
    if row.Country == 'USA':
        
        # Get paths, etc
        basins_path = Path(data_path) / cs_basin_folder
        site,_, raw_path,_ = cs.prepare_flow_download_outputs(cs_meta, i, basins_path)
        
        # Construct the download URL
        url = f'{main_url}?format=rdb&sites={site}&startDT={time_s}&endDT={time_e}&parameterCd={var}&siteStatus=all'
        
        # Open the URL with pandas, skipping the header comments (#); low_mem prevents mixed datatype warning 
        df = pd.read_csv(url, delimiter='\t', comment='#', low_memory=False) 
        
        # Strip any non-data lines: these don't specify a site ID
        df = df[df.site_no == site]
        
        # To file
        df.to_csv(raw_path, index=False)
        print(f'Succesfully downloaded data for USA station {site}')        

1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226


In [ ]:
cs_meta['flow_obs_availability_start']
cs_meta['flow_obs_availability_end']
cs_meta['flow_obs_availability_resolution']

In [5]:
cs_meta

,Country,Station_id,Station_name,Station_lat,Station_lon,Station_source,Outlet_lat,Outlet_lon,Outlet_source,Mapped_lat,...,Basin_area_km2,Ref_area_1_src,Ref_area_1_km2,Ref_area_2_src,Ref_area_2_km2,Ref_shape,Ref_shape_source,Ref_shape_area_km2,Ref_and_new_shape_fractional_overlap,Manual_delineation_notes
0,CAN,01AD002,SAINT JOHN RIVER AT FORT KENT,47.25806,-68.59583,WSC 20222 data set,47.257877,-68.594919,WSC 20222 data set,47.257312,...,-999,HYDAT gross drainage area,14700.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,14677.400000,-1,NaN
1,CAN,01AD003,ST. FRANCIS RIVER AT OUTLET OF GLASIER LAKE,47.20661,-68.95694,WSC 20222 data set,47.206016,-68.956727,WSC 20222 data set,47.205895,...,-999,HYDAT gross drainage area,1350.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,1348.300000,-1,NaN
2,CAN,01AE001,FISH RIVER NEAR FORT KENT,47.23750,-68.58278,WSC 20222 data set,47.237438,-68.581991,WSC 20222 data set,47.237438,...,-999,HYDAT gross drainage area,2260.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,2238.950000,-1,NaN
3,CAN,01AF007,GRANDE RIVIERE AT VIOLETTE BRIDGE,47.24697,-67.92128,WSC 20222 data set,47.246908,-67.921271,WSC 20222 data set,47.246667,...,-999,HYDAT gross drainage area,339.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,325.196000,-1,NaN
4,CAN,01AF009,IROQUOIS RIVER AT MOULIN MORNEAULT,47.45783,-68.35683,WSC 20222 data set,47.457622,-68.356710,WSC 20222 data set,47.457622,...,-999,HYDAT gross drainage area,182.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,183.430000,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1693,USA,14309500,"WEST FORK COW CREEK NEAR GLENDALE, OR",42.80400,-123.61091,CAMELS-US,-999.000000,-999.000000,NaN,42.804462,...,-999,GAGES II area,224.92,Geospatial fabric area,226.31,yes,CAMELS-US data set (HCDN),226.314346,-1,NaN
1694,USA,14316700,"STEAMBOAT CREEK NEAR GLIDE, OR",43.34984,-122.72894,CAMELS-US,-999.000000,-999.000000,NaN,43.349840,...,-999,GAGES II area,587.90,Geospatial fabric area,588.01,yes,CAMELS-US data set (HCDN),588.025033,-1,NaN
1695,USA,14325000,"SOUTH FORK COQUILLE RIVER AT POWERS, OR",42.89150,-124.07065,CAMELS-US,-999.000000,-999.000000,NaN,42.892325,...,-999,GAGES II area,443.07,Geospatial fabric area,444.92,yes,CAMELS-US data set (HCDN),444.925651,-1,NaN
1696,USA,14362250,"STAR GULCH NEAR RUCH, OR.",42.15401,-123.07532,CAMELS-US,-999.000000,-999.000000,NaN,42.155000,...,-999,GAGES II area,41.42,Geospatial fabric area,43.88,yes,CAMELS-US data set (HCDN),43.877901,-1,NaN


In [34]:
import pytz

In [41]:
est = pytz.timezone('EST')

In [24]:
test = df.copy()

In [26]:
test['time_tmp'] = pd.to_datetime(test.datetime)

In [42]:
test.time_tmp.dt.tz_localize(est)

1        1989-10-01 01:30:00-05:00
2        1989-10-01 02:00:00-05:00
3        1989-10-01 02:30:00-05:00
4        1989-10-01 03:00:00-05:00
5        1989-10-01 03:30:00-05:00
                    ...           
131067   1995-05-07 23:15:00-05:00
131068   1995-05-07 23:30:00-05:00
131069   1995-05-07 23:45:00-05:00
131070   1995-05-08 00:00:00-05:00
131071   1995-05-08 00:15:00-05:00
Name: time_tmp, Length: 131071, dtype: datetime64[ns, EST]

In [ ]:
# df[tz] = lambda(something; if 'EST', use EST)

In [25]:
pd.to_datetime(test.datetime).dt.tz_localize(test.tz_cd)

TypeError: <class 'pandas.core.series.Series'>

In [ ]:
# Convert the datetime
pd.to_datetime()

In [20]:
df

,agency_cd,site_no,datetime,tz_cd,40346_00060,40346_00060_cd
0,5s,15s,20d,6s,14n,10s
1,USGS,02395120,1989-10-01 01:30,EDT,864,A:[91]
2,USGS,02395120,1989-10-01 02:00,EDT,905,A:[91]
3,USGS,02395120,1989-10-01 02:30,EDT,973,A:[91]
4,USGS,02395120,1989-10-01 03:00,EDT,1030,A:[91]
...,...,...,...,...,...,...
1080232,USGS,2395120,2023-01-02 22:45,EST,18.4,A
1080233,USGS,2395120,2023-01-02 23:00,EST,18.4,A
1080234,USGS,2395120,2023-01-02 23:15,EST,18.4,A
1080235,USGS,2395120,2023-01-02 23:30,EST,18.4,A
